In [8]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from requests_html import HTML
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.common.exceptions import WebDriverException
from tqdm import tqdm
import pymysql
import datetime
import requests
import os

In [9]:
#Congiruando o driver
options = Options()
options.add_argument("--headless")
options.add_argument('--disable-gpu')
options.add_argument("--log-level=3")
options.add_argument('--no-sandbox')
options.add_experimental_option('useAutomationExtension', False)
options.add_experimental_option('excludeSwitches', ['enable-logging'])

In [10]:
driver = webdriver.Chrome(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Selenium\Selenium_103", options=options)

C:\Users\pedro\AppData\Local\Temp\ipykernel_28248\1319122002.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Selenium\Selenium_103", options=options)


In [11]:
Zoom_Links = []

In [12]:
def getting_n_creating_zoom_urls(brand):
    connection = pymysql.connect(host='mysqlserver.cnzboqhfvndh.sa-east-1.rds.amazonaws.com',
                                 user='admin',
                                 password='turtle316712',
                                 database='Products_Brands',
                                 cursorclass=pymysql.cursors.DictCursor)

    #Criando o caminho do Databae
    c = connection.cursor()

    #Criando a Query
    Sql_query = "SELECT * FROM Products WHERE Brand = '%s'" % (brand)

    #Conectando com o banco de dados
    c.execute(Sql_query)
    result = c.fetchall()
    c.close()
    connection.close()

    #Passando todos o dataframe para Lowercase
    df = pd.DataFrame()
    df['Brand'] = [item['Brand'] for item in result]
    df['Product_Name'] = [item['Name'] for item in result]

    #Arrumando espaços vazios
    # Arrumano os espaços vazios
    df['Product_Name'] = df['Product_Name'].str.replace(" ", "%20")
    df['Product_Name'] = df['Product_Name'].str.replace("-", "%20")

    # Criando uma nova coluna no database com a formatação certa
    df['Urls'] = df['Brand'] + "%20" + df['Product_Name']

    # Criando a nova coluna que são as urls de pesquisa
    df['Urls_search'] = "https://www.zoom.com.br/search?q=" + df['Urls']

    return df

In [13]:
Df_Products = getting_n_creating_zoom_urls('Wacom')
Df_Products

,Brand,Product_Name,Urls,Urls_search
0,Wacom,CTL472,Wacom%20CTL472,https://www.zoom.com.br/search?q=Wacom%20CTL472
1,Wacom,CTL4100,Wacom%20CTL4100,https://www.zoom.com.br/search?q=Wacom%20CTL4100
2,Wacom,CTL4100WLE0,Wacom%20CTL4100WLE0,https://www.zoom.com.br/search?q=Wacom%20CTL41...
3,Wacom,CTL4100WLK0,Wacom%20CTL4100WLK0,https://www.zoom.com.br/search?q=Wacom%20CTL41...
4,Wacom,CTL6100WLK0,Wacom%20CTL6100WLK0,https://www.zoom.com.br/search?q=Wacom%20CTL61...
5,Wacom,DTC133W0A,Wacom%20DTC133W0A,https://www.zoom.com.br/search?q=Wacom%20DTC13...
6,Wacom,PTH460,Wacom%20PTH460,https://www.zoom.com.br/search?q=Wacom%20PTH460
7,Wacom,DTK1660K0A,Wacom%20DTK1660K0A,https://www.zoom.com.br/search?q=Wacom%20DTK16...
8,Wacom,PTH660,Wacom%20PTH660,https://www.zoom.com.br/search?q=Wacom%20PTH660
9,Wacom,CTL6100WLE0,Wacom%20CTL6100WLE0,https://www.zoom.com.br/search?q=Wacom%20CTL61...


In [14]:
def get_links(url):
    global  Zoom_Links

    time.sleep(10)
    response = requests.get(url)
    html = response.text

    Soup = BeautifulSoup(html, 'html.parser')
    for a in Soup.find_all("a", href=True):
        Zoom_Links.append(a['href'])

    Zoom_Links = [s for s in Zoom_Links if 'searchterm' in s]

In [15]:
for url in tqdm(Df_Products['Urls_search']):
    get_links(url)


100%|██████████| 10/10 [01:47<00:00, 10.74s/it]


In [16]:
print(len(Zoom_Links))

63


In [31]:
Zoom_Links_with_https = [s for s in Zoom_Links if 'https://' in s]
Zoom_Links_without_https = [s for s in Zoom_Links if not 'https://' in s]

print(len(Zoom_Links_with_https))
print(len(Zoom_Links_without_https))

print(int(len(Zoom_Links)))

37
26
63


In [36]:
def creating_the_dataframe(urls):
    Df = pd.DataFrame()
    Df['Urls'] = urls
    Df['Beginning'] = Df['Urls'].str.replace("https://www.zoom.com.br","")
    Df['Urls_search'] = "https://www.zoom.com.br" + Df['Beginning']

    return Df

In [37]:
Df_urls = creating_the_dataframe(Zoom_Links)
Df_urls

C:\Users\pedro\AppData\Local\Temp\ipykernel_28248\2409044783.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  Df['Beginning'] = Df['Urls'].str.replace("https://www.zoom.com.br","")


,Urls,Beginning,Urls_search
0,/mesa-digitalizadora/mesa-digitalizadora-wacom...,/mesa-digitalizadora/mesa-digitalizadora-wacom...,https://www.zoom.com.br/mesa-digitalizadora/me...
1,/mesa-digitalizadora/wacom-one-ctl472-mesa-dig...,/mesa-digitalizadora/wacom-one-ctl472-mesa-dig...,https://www.zoom.com.br/mesa-digitalizadora/wa...
2,/mesa-digitalizadora/mesa-digitalizadora-one-b...,/mesa-digitalizadora/mesa-digitalizadora-one-b...,https://www.zoom.com.br/mesa-digitalizadora/me...
3,/mesa-digitalizadora/mesa-digitalizadora-one-b...,/mesa-digitalizadora/mesa-digitalizadora-one-b...,https://www.zoom.com.br/mesa-digitalizadora/me...
4,https://www.zoom.com.br/lead?oid=788011728&cha...,/lead?oid=788011728&channel=86&index=4&searcht...,https://www.zoom.com.br/lead?oid=788011728&cha...
...,...,...,...
58,/mesa-digitalizadora/mesa-digitalizadora-wacom...,/mesa-digitalizadora/mesa-digitalizadora-wacom...,https://www.zoom.com.br/mesa-digitalizadora/me...
59,/mesa-digitalizadora/folha-textura-aspera-para...,/mesa-digitalizadora/folha-textura-aspera-para...,https://www.zoom.com.br/mesa-digitalizadora/fo...
60,/mesa-digitalizadora/mesa-digitalizadora-wacom...,/mesa-digitalizadora/mesa-digitalizadora-wacom...,https://www.zoom.com.br/mesa-digitalizadora/me...
61,/mesa-digitalizadora/mesa-digitalizadora-wacom...,/mesa-digitalizadora/mesa-digitalizadora-wacom...,https://www.zoom.com.br/mesa-digitalizadora/me...


In [42]:
Zoom_Price = []
Zoom_Seller = []
Zoom_Installment = []
Zoom_Title = []
Zoom_Urls = []

In [43]:
def Get_atribuites(url):
    time.sleep(20)
    response = driver.get(url)
    time.sleep(5)
    print(url)


    try:
        driver.find_element(by=By.CLASS_NAME, value="Button_Button__GLsxI Button_secondary__lv0si Button_medium__ea4jW Button_mediumAtSmall__x_KxA Button_largeAtLarge__5iZss SeeMore_Button__fveoe").click()
    except:
        pass

    try:
        driver.find_element(by=By.CLASS_NAME, value="Button_Button__GLsxI Button_secondary__lv0si Button_medium__ea4jW Button_mediumAtSmall__x_KxA Button_largeAtLarge__5iZss SeeMore_Button__fveoe").click()
    except:
        pass

    try:
        driver.find_element(by=By.CLASS_NAME, value="Button_Button__GLsxI Button_secondary__lv0si Button_medium__ea4jW Button_mediumAtSmall__x_KxA Button_largeAtLarge__5iZss SeeMore_Button__fveoe").click()
    except:
        pass

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')


    Soup = BeautifulSoup(html_str, 'html.parser')

    #price = Soup.find(class_='Text_Text__bOTfK Text_DesktopHeadingM__pBPKa').text
    try:
        title = Soup.find(class_='Text_Text__bOTfK Text_MobileSubtitleM__4Op9D Text_DesktopSubtitleMAtLarge__7Nr81').text
    except:
        title = 'erro'
    #Installment = Soup.find(class_='Price_Installments__8hwMt').text
    #N_offers = Soup.find(class_='Anchor_Anchor__1FNBP').text

    #print(price + '\n' + title + "\n" + Installment + "\n" + N_offers)
    try:
        for a in Soup.find_all(class_='Text_Text__bOTfK Text_MobileHeadingM__FRgbv'):
            Zoom_Price.append(a.text)
            Zoom_Title.append(title)
            Zoom_Urls.append(url)
    except:
        Zoom_Price.append("ERRO")
        Zoom_Title.append("ERRO")

    try:
        for b in Soup.find_all(class_='Text_Text__bOTfK Text_MobileLabelXs__rr7ZF OfferPrice_Installments__006GM'):
            Zoom_Installment.append(b.text)
    except:
        Zoom_Installment.append("erro")

    try:
        for c in Soup.find_all(class_='OfferMerchant_Merchant__1ZgkP'):
            Zoom_Seller.append(c.text)
    except:
        Zoom_Seller.append('Erro')


In [44]:
for url in tqdm(Df_urls['Urls_search']):
    Get_atribuites(url)

  2%|▏         | 1/63 [00:25<26:37, 25.77s/it]

https://www.zoom.com.br/mesa-digitalizadora/mesa-digitalizadora-wacom-one-by-wacom-ctl472?_lc=88&searchterm=Wacom%20CTL472
https://www.zoom.com.br/mesa-digitalizadora/wacom-one-ctl472-mesa-digitalizadora-preto-vermelho?_lc=88&searchterm=Wacom%20CTL472


  5%|▍         | 3/63 [01:17<25:50, 25.85s/it]

https://www.zoom.com.br/mesa-digitalizadora/mesa-digitalizadora-one-by-wacom-ctl472-pequena?_lc=88&searchterm=Wacom%20CTL472


  6%|▋         | 4/63 [01:43<25:21, 25.78s/it]

...

https://www.zoom.com.br/mesa-digitalizadora/mesa-digitalizadora-wacom-intuos-pro-paper-edition-pth660p?_lc=88&searchterm=Wacom%20PTH660


 98%|█████████▊| 62/63 [27:01<00:25, 25.84s/it]

https://www.zoom.com.br/mesa-digitalizadora/mesa-digitalizadora-wacom-media-bluetooth-verde-intuos-ctl6100wle0?_lc=88&searchterm=Wacom%20CTL6100WLE0


100%|██████████| 63/63 [27:26<00:00, 26.14s/it]

https://www.zoom.com.br/mesa-digitalizadora/mesa-digitalizadora-wacom-intuos-m-bluetooth-pistache-ctl6100wle0?_lc=88&searchterm=Wacom%20CTL6100WLE0


In [48]:
Zoom_Installment.append("Erro")

In [71]:
Df_final = pd.DataFrame()
Df_final['Urls'] = Zoom_Urls
Df_final['MARKETPLACE'] = 'ZOOM'
Df_final['SELLER'] = Zoom_Seller
Df_final['PRICE'] = Zoom_Price
Df_final['PRICE'] = Df_final['PRICE'].str.replace("à vista","")
Df_final['PRICE'] = Df_final['PRICE'].str.replace("R$ ","",regex=False)

Df_final['INSTALLMENT_FULL'] = Zoom_Installment
Df_final['PARCEL'] = Df_final['INSTALLMENT_FULL'].str.partition("ou ")[2].str.partition("x")[0]

Df_final['INSTALLMENT'] = Df_final['INSTALLMENT_FULL'].str.partition("R$ ")[2].str.partition(" sem juros")[0]


Df_final['PRODUCT'] = Zoom_Title

Df_final

,Urls,MARKETPLACE,SELLER,PRICE,INSTALLMENT_FULL,PARCEL,INSTALLMENT,PRODUCT
0,https://www.zoom.com.br/mesa-digitalizadora/me...,ZOOM,Casas Bahia,"341,45","ou 7x de R$ 48,78",7,"48,78",Mesa Digitalizadora Wacom One By Wacom - CTL472
1,https://www.zoom.com.br/mesa-digitalizadora/me...,ZOOM,Magazine Luiza,"219,90","ou 4x de R$ 64,68",4,"64,68",Mesa Digitalizadora Wacom One By Wacom - CTL472
2,https://www.zoom.com.br/mesa-digitalizadora/me...,ZOOM,Inpower Eletrônicos,"333,00","ou 6x de R$ 55,50 sem juros",6,"55,50",Mesa Digitalizadora Wacom One By Wacom - CTL472
3,https://www.zoom.com.br/mesa-digitalizadora/me...,ZOOM,Extra,"350,10","ou 4x de R$ 87,52 sem juros",4,"87,52",Mesa Digitalizadora Wacom One By Wacom - CTL472
4,https://www.zoom.com.br/mesa-digitalizadora/me...,ZOOM,Ponto,"350,10","ou 4x de R$ 87,52 sem juros",4,"87,52",Mesa Digitalizadora Wacom One By Wacom - CTL472
5,https://www.zoom.com.br/mesa-digitalizadora/me...,ZOOM,Oficina dos Bits,"351,24","ou 6x de R$ 58,54 sem juros",6,"58,54",Mesa Digitalizadora Wacom One By Wacom - CTL472
6,https://www.zoom.com.br/mesa-digitalizadora/me...,ZOOM,MService,"499,00","ou 6x de R$ 83,17",6,"83,17",Mesa Digitalizadora Wacom One By Wacom - CTL472
7,https://www.zoom.com.br/mesa-digitalizadora/me...,ZOOM,Casas Bahia,"449,00","ou 9x de R$ 49,89",9,"49,89",Mesa Digitalizadora Wacom One By Wacom - CTL472
8,https://www.zoom.com.br/mesa-digitalizadora/me...,ZOOM,Magazine Luiza,"290,67","ou 5x de R$ 64,59",5,"64,59",Mesa Digitalizadora Wacom One By Wacom - CTL472
9,https://www.zoom.com.br/mesa-digitalizadora/me...,ZOOM,Ponto,"503,53","ou 10x de R$ 50,35 sem juros",10,"50,35",Mesa Digitalizadora Wacom One By Wacom - CTL472


In [72]:
Df_final.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Software_Artemis\Data\Brands_Downloads\Wacom\Zoom_Wacom.xlsx", index=False)

In [6]:
url = "https://www.zoom.com.br/search?q=wacom%20ctl%20472"

response = requests.get(url)
print(response.status_code)

200


In [7]:
html = response.text
Soup = BeautifulSoup(html, 'html.parser')

In [8]:
links = []
for a in Soup.find_all("a",href=True):
    links.append(a['href'])



In [10]:
links = [s for s in links if 'searchterm' in s]

In [11]:
links

['/mesa-digitalizadora/mesa-digitalizadora-one-by-wacom-pequena-2048-pontos-de-pressao-2540-ipi-21x14-6-cm-ctl-472l?_lc=88&searchterm=wacom%20ctl%20472',
 '/mesa-digitalizadora/mesa-digitalizadora-wacom-one-by-wacom-ctl472?_lc=88&searchterm=wacom%20ctl%20472',
 '/mesa-digitalizadora/wacom-one-ctl472-mesa-digitalizadora-preto-vermelho?_lc=88&searchterm=wacom%20ctl%20472',
 '/mesa-digitalizadora/mesa-digitalizadora-one-by-wacom-ctl472-pequena?_lc=88&searchterm=wacom%20ctl%20472',
 '/mesa-digitalizadora/mesa-digitalizadora-one-by-wacom-ctl472?_lc=88&searchterm=wacom%20ctl%20472',
 'https://www.zoom.com.br/lead?oid=788011728&channel=86&index=5&searchterm=wacom%20ctl%20472',
 '/lead?oid=350461845&channel=86&index=6&searchterm=wacom%20ctl%20472',
 'https://www.zoom.com.br/lead?oid=536360373&channel=86&index=7&searchterm=wacom%20ctl%20472',
 '/lead?oid=852417963&channel=86&index=8&searchterm=wacom%20ctl%20472',
 'https://www.zoom.com.br/lead?oid=799183152&channel=86&index=9&searchterm=wacom%2

In [12]:
print(len(links))

17


In [9]:
https://www.zoom.com.br/mesa-digitalizadora/mesa-digitalizadora-one-by-wacom-pequena-2048-pontos-de-pressao-2540-ipi-21x14-6-cm-ctl-472l?_lc=88&searchterm=wacom%20ctl%20472

SyntaxError: invalid decimal literal (4051688236.py, line 1)

In [10]:
response = requests.get("https://www.zoom.com.br/mesa-digitalizadora/mesa-digitalizadora-one-by-wacom-pequena-2048-pontos-de-pressao-2540-ipi-21x14-6-cm-ctl-472l?_lc=88&searchterm=wacom%20ctl%20472")
html = response.text

Soup = BeautifulSoup(html, 'html.parser')

In [15]:
print(Soup.find(class_='Text_Text__bOTfK Text_MobileSubtitleM__4Op9D Text_DesktopSubtitleMAtLarge__7Nr81').text)
print(Soup.find(class_='Text_Text__bOTfK Text_DesktopHeadingM__pBPKa').text)
print(Soup.find(class_='Anchor_Anchor__1FNBP').text)

Mesa Digitalizadora One by Wacom Pequena 2048 Pontos de Pressão 2540 ipi 21x14,6 cm - CTL-472L
R$ 339,53
Ver esta e mais 1 oferta


In [16]:
response = requests.get("https://www.buscape.com.br/mesa-digitalizadora/mesa-digitalizadora-wacom-one-by-wacom-ctl472?_lc=88&searchterm=wacom%20ctl%20472")
html = response.text

Soup = BeautifulSoup(html, 'html.parser')

In [17]:
print(Soup.find(class_='Text_Text__bOTfK Text_MobileSubtitleM__4Op9D Text_DesktopSubtitleMAtLarge__7Nr81').text)
print(Soup.find(class_='Text_Text__bOTfK Text_DesktopHeadingM__pBPKa').text)
print(Soup.find(class_='Anchor_Anchor__1FNBP').text)

Mesa Digitalizadora Wacom One By Wacom - CTL472
R$ 219,90
Ver esta e mais 44 ofertas
